In [ ]:
import os

# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from backend.cycle_preference_inconsistency import get_edges_list, graph_from_edges, inconsistencies_for_queryset
import networkx as nx
from backend.rating_fields import VIDEO_FIELDS, MAX_VALUE
import numpy as np
from backend.models import DjangoUser
from django.db.models import Value, F, FloatField, Func, Q
import shortuuid
import string
from matplotlib import pyplot as plt
from backend.cycle_preference_inconsistency import inconsistencies_3_for_queryset,\
get_cycles_weights_3, get_edges_list_db
from backend.tests.tests import TestInconsistencies3
add_inconsistency = TestInconsistencies3.add_inconsistency

In [ ]:
def random_alphanumeric(length=10, alphabet=None):
    """UUID1 without -."""
    if alphabet is None:
        alphabet = string.ascii_lowercase + string.digits
    res = shortuuid.ShortUUID(alphabet=alphabet).random(length=length)
    return str(res).replace('-', '')

In [ ]:
un = 'inconsistent_person'
DjangoUser.objects.filter(username=un).delete()
dj_u = DjangoUser.objects.create_user(username=un)

In [ ]:
user, _ = UserPreferences.objects.get_or_create(user=dj_u)

In [ ]:
n_videos = 10000
n_ratings = 10000

In [ ]:
Video.objects.filter(video_id__endswith='-inc').delete()

In [ ]:
video_ids = ["%09d-%s-inc" % (i, random_alphanumeric()) for i in range(n_videos)]

In [ ]:
objs = [Video(video_id=vid) for vid in video_ids]

In [ ]:
_ = Video.objects.bulk_create(objs)

In [ ]:
videos = list(Video.objects.filter(video_id__in=video_ids))

In [ ]:
# creating random ratings
video_firsts = np.random.choice(n_videos - 1, n_ratings, replace=True) + 1
video_seconds = []
for i in video_firsts:
    video_seconds.append(np.random.choice(i))
assert len(video_firsts) == len(video_seconds)
pairs = list(zip(video_firsts, video_seconds))

assert all([x > y] for x, y in pairs)
assert (len(pairs) - len(set(pairs))) / len(pairs) < 0.05
pairs = set(pairs)

In [ ]:
ratings = [ExpertRating(user=user, video_1=videos[int(i)], video_2=videos[int(j)],
                        **{f: np.random.rand() * MAX_VALUE for f in VIDEO_FIELDS}
                       ) for i, j in pairs]

In [ ]:
_ = ExpertRating.objects.bulk_create(ratings)

In [ ]:
n_inc = 10
ground_truth_inconsistencies = [add_inconsistency(user, feature='reliability') for _ in range(n_inc)]

In [ ]:
qs = ExpertRating.objects.filter(user=user)

In [ ]:
ground_truth_inconsistencies

In [ ]:
edges = get_edges_list_db(qs, 'reliability')

In [ ]:
len(edges)

In [ ]:
%timeit get_cycles_weights_3(edges)

In [ ]:
cycles_3, weights_3 = get_cycles_weights_3(edges)

In [ ]:
assert set(ground_truth_inconsistencies).issubset(cycles_3)

In [ ]:
inc_founds = inconsistencies_3_for_queryset(user)

for (v1id, v2id, v3id) in ground_truth_inconsistencies:
    v1 = Video.objects.get(pk=v1id)
    v2 = Video.objects.get(pk=v2id)
    v3 = Video.objects.get(pk=v3id)
    
    yt_ids = [v1.video_id, v2.video_id, v3.video_id]
    
    founds = []
    for inc_found in inc_founds:
        if inc_found['feature'] != 'reliability':
            continue
        if set(yt_ids) != set(inc_found['videos']):
            continue
        founds.append(inc_found)
    assert len(founds) == 1, (founds, yt_ids)
    
    inc_found = founds[0]
    
    cmp = inc_found['comparisons']
    assert cmp[0]['videoA'] == yt_ids[0], (cmp, yt_ids)
    assert cmp[0]['videoB'] == yt_ids[1], (cmp, yt_ids)
    assert cmp[0]['score'] == 0.0
    
    assert cmp[1]['videoA'] == yt_ids[1], (cmp, yt_ids)
    assert cmp[1]['videoB'] == yt_ids[2], (cmp, yt_ids)
    assert cmp[1]['score'] == 0.0
    
    assert cmp[2]['videoA'] == yt_ids[2], (cmp, yt_ids)
    assert cmp[2]['videoB'] == yt_ids[0], (cmp, yt_ids)
    assert cmp[2]['score'] == 0.0
    
    assert inc_found['videos'] == yt_ids + [yt_ids[0]]
    assert inc_found['weight'] == 0.5

In [ ]:
inc_found

In [ ]:
inconsistencies_3_for_queryset(user)

In [ ]:
%timeit inconsistencies_3_for_queryset(user)

In [ ]:
cycles_3, weights_3

In [ ]:
ground_truth_inconsistencies

In [ ]:
edges = get_edges_list_db(qs, VIDEO_FIELDS[0])

In [ ]:
%timeit get_edges_list_db(qs, VIDEO_FIELDS[0])

In [ ]:
%timeit get_edges_list(qs, VIDEO_FIELDS[0])

In [ ]:
len(get_edges_list(qs, VIDEO_FIELDS[0])), len(get_edges_list_db(qs, VIDEO_FIELDS[0]))

In [ ]:
edges = get_edges_list(qs, VIDEO_FIELDS[0])

In [ ]:
%timeit graph_from_edges(edges)

In [ ]:
%timeit graph_from_edges(get_edges_list(qs, VIDEO_FIELDS[0]))

In [ ]:
qs[0].reliability

In [ ]:
G = graph_from_edges(get_edges_list(qs, VIDEO_FIELDS[0]))

In [ ]:
nx.draw(G, with_labels=False, font_weight='bold')

In [ ]:
inconsistencies_for_queryset(user)

In [ ]:
%timeit inconsistencies_for_queryset(UserPreferences.objects.get(user__username='lenhoang'))